In [2]:
import argparse
import ntpath
import numpy as np
import pandas as pd
import json
from glob import glob
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import os

In [4]:
checkpoint_path = 'exp/mix_smpl_w_pw3d/hybrik_transformer_smpl24_w_pw3d.yaml-hybrik_transformer_blocks6_layers3_lw512_dp0.0_inv_sqrt_lr0.0005_aug_w_pw3d'
checkpoint_path = 'exp/mix_smpl/hybrik_transformer_smpl24_hrnet.yaml-hybrik_transformer_warmup_4000_hrnet_bs56_lr0.0005'

def make_checkpoint_average(path: str, epoch_min: int, epoch_max: int, prefix: str='average_model'):
    checkpoint_pattern = os.path.join(path, 'model_*')
    checkpoints = glob(checkpoint_pattern)
    
    filtered_checkpoints = []
    epoch_actual_min = np.Inf
    epoch_actual_max = 0
    for c in checkpoints:
        epoch = int(os.path.basename(c).split('model_')[1].split('.pth')[0])
        if (epoch >= epoch_min) and (epoch <= epoch_max):
            epoch_actual_min = min(epoch_actual_min, epoch)
            epoch_actual_max = max(epoch_actual_max, epoch)
            filtered_checkpoints.append(c)
    
    m_ave = torch.load(filtered_checkpoints[0])
    for c in tqdm(filtered_checkpoints[1:]):
        m = torch.load(c)

        for k, v in m_ave.items():
            m_ave[k] = v + m[k]

    for k, v in m_ave.items():
        m_ave[k] = v / len(filtered_checkpoints)
        
    save_path = os.path.join(checkpoint_path, f"{prefix}_{epoch_actual_min}_{epoch_actual_max}.pth")
    print(f"Saving model {save_path}")
    torch.save(m_ave, save_path)
    
    return filtered_checkpoints
    

filtered_checkpoints = make_checkpoint_average(checkpoint_path, 180, 200, prefix='average_model_hrnet')


Saving model exp/mix_smpl/hybrik_transformer_smpl24_hrnet.yaml-hybrik_transformer_warmup_4000_hrnet_bs56_lr0.0005/average_model_hrnet_181_199.pth
